In [1]:
from datetime import date
from datetime import timedelta

import pandas as pd
import numpy as np

import yfinance as yf

from scipy.stats import norm

from pypfopt import expected_returns 
from pypfopt import risk_models 
from pypfopt import plotting
from pypfopt import objective_functions

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import DiscreteAllocation, get_latest_prices
from pypfopt.base_optimizer import BaseConvexOptimizer
from pypfopt import DiscreteAllocation, get_latest_prices

import warnings
warnings.filterwarnings('ignore')

In [2]:
end = date.today() - timedelta(days = 1)
start = date(end.year-5,end.month,end.day)

In [3]:
stock_list_50 = []
stock_symbols_50 = []

df_50 = pd.read_csv('../data/ind_nifty50list.csv')
stock_list_50 = df_50['Symbol'].to_list()
for stock in stock_list_50:
    stock_symbols_50.append(stock+'.NS')

stock_list_100 = []
stock_symbols_100 = []
    
df_100 = pd.read_csv('../data/ind_niftymidcap100list.csv')
stock_list_100 = df_100['Symbol'].to_list()
for stock in stock_list_100:
    stock_symbols_100.append(stock+'.NS')

In [4]:
for stock, stock_symbol in zip(stock_list_50, stock_symbols_50):
    globals()[stock] = yf.download(stock_symbol, start, end)
    globals()[stock].dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
for stock, stock_symbol in zip(stock_list_100, stock_symbols_100):
    globals()[stock] = yf.download(stock_symbol, start, end)
    globals()[stock].dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
nifty_50 = yf.download("^NSEI", start, end)

nifty_50_daily = nifty_50['Adj Close'].pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


In [7]:
closing_df_50 = pd.DataFrame(index=WIPRO.index, columns=stock_list_50)
for stock in stock_list_50:
    closing_df_50[stock] = globals()[stock]['Adj Close'] 

closing_df_100 = pd.DataFrame(index=ATGL.index, columns=stock_list_100)    
for stock in stock_list_100:
    closing_df_100[stock] = globals()[stock]['Adj Close'] 

In [8]:
dict_50 = dict.fromkeys(stock_list_50, "nifty50")
dict_100 = dict.fromkeys(stock_list_100, "nifty100")

In [9]:
sl = stock_list_50 + stock_list_100

sector_mapper = dict(dict_50)
sector_mapper.update(dict_100)

sector_lower_1 = {"nifty50": 0.5, "nifty100": 0.5}
sector_upper_1 = {"nifty50": 0.51, "nifty100": 0.51}

sector_lower_2 = {"nifty50": 0.3, "nifty100": 0.7}
sector_upper_2 = {"nifty50": 0.31, "nifty100": 0.71}

In [10]:
df = pd.DataFrame(index=WIPRO.index, columns=sl)
for stock in sl:
    df[stock] = globals()[stock]['Adj Close']

In [11]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.CovarianceShrinkage(df).ledoit_wolf(shrinkage_target='single_factor')

ef1 = EfficientFrontier(expected_returns=mu, cov_matrix=S)
ef1.add_sector_constraints(sector_mapper,sector_lower_1, sector_upper_1)
weights1 = ef1.max_sharpe()
cleaned_weights1 = ef1.clean_weights()
ef1.portfolio_performance(verbose=True, risk_free_rate=0.0602)

print('\n')

ef2 = EfficientFrontier(expected_returns=mu, cov_matrix=S)
ef2.add_sector_constraints(sector_mapper,sector_lower_2, sector_upper_2)
weights2 = ef2.max_sharpe()
cleaned_weights2 = ef2.clean_weights()
ef2.portfolio_performance(verbose=True, risk_free_rate=0.0602)

Expected annual return: 79.6%
Annual volatility: 15.9%
Sharpe Ratio: 4.88


Expected annual return: 96.1%
Annual volatility: 17.4%
Sharpe Ratio: 5.42


(0.9614271758140537, 0.1736037164203662, 5.422851510473833)

In [12]:
cw1 = {x:y for x,y in cleaned_weights1.items() if y!=0}
len(cw1)

15

In [13]:
cw2 = {x:y for x,y in cleaned_weights2.items() if y!=0}
len(cw2)

12

In [14]:
# REDUCING STOCK NUMBER TO 10

In [15]:
def stock_10(df, gamma):
    
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.CovarianceShrinkage(df).ledoit_wolf(shrinkage_target='single_factor')
    
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg, gamma=gamma)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    cleaned_weights = {x:y for x,y in cleaned_weights.items() if y!=0}
    
    return cleaned_weights

In [16]:
new_cw1 = {}
gamma = 0.0005
while(len(new_cw1) < 10):
    new_df_1 = df[cw1].copy()
    new_cw1 = stock_10(new_df_1, gamma) 
    gamma += 0.0005
len(new_cw1)

10

In [17]:
new_cw2 = {}
gamma = 0.0005
while(len(new_cw2) < 10):
    new_df_2 = df[cw2].copy()
    new_cw2 = stock_10(new_df_2, gamma)
    gamma += 0.0005
len(new_cw2)

10

In [18]:
# METRICS CALCULATIONS

In [19]:
cw = list(set(list(new_cw1.keys()) + list(new_cw2.keys())))

tdf = df[cw].copy().pct_change()
tdf.drop([tdf.index[0]], inplace=True)
tcs = tdf.copy()
tcs['nifty_50'] = nifty_50_daily
tcsdf = tcs.corr(method='pearson')

tdf1 = df[new_cw1].copy().pct_change()
tdf1.drop([tdf1.index[0]], inplace=True)
tcs1 = tdf1.copy()
tcs1['nifty_50'] = nifty_50_daily
tcsdf1 = tcs1.corr(method='pearson')

tdf2 = df[new_cw2].copy().pct_change()
tdf2.drop([tdf2.index[0]], inplace=True)
tcs2 = tdf2.copy()
tcs2['nifty_50'] = nifty_50_daily
tcsdf2 = tcs2.corr(method='pearson')

In [20]:
stock_beta = pd.Series(index=cw)
nifty_std = tcs['nifty_50'].std()

for stock in cw:
    stock_beta[stock] = (tcsdf['nifty_50'][stock] * tcs[stock].std()) / nifty_std

In [21]:
metrics = ['Return','Volatility','Sharpe Ratio', 'VaR', 'Portfolio Beta', 'Treynor Ratio', 'Jensen Alpha']
metrics_df = pd.DataFrame(columns=metrics)

In [22]:
risk_free_rate = 0.0602
alpha = 0.01
days = 252
market_return = nifty_50_daily.mean() * days 

In [23]:
# P1

r1 = np.zeros(len(metrics))
sb1 = stock_beta[new_cw1].copy()
pb1 = 0
twl1 = np.array(list(new_cw1.values()))
tcovdf1 = tdf1.cov()

pr1 = np.sum(tdf1.mean() * twl1) * days
pstd1 = np.sqrt(np.dot(twl1.T,np.dot(tcovdf1, twl1))) * np.sqrt(days)

r1[0] = pr1
r1[1] = pstd1

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r1[2] = (r1[0] - risk_free_rate) / r1[1]

# VaR => return - (Z * volatility)
r1[3] = abs(pr1 - (pstd1 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl1)):
    pb1 += twl1[j] * sb1[j]
r1[4] = pb1

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r1[5] = (r1[0] - risk_free_rate) / r1[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r1[6] = r1[0] - (risk_free_rate + r1[4] * (market_return - risk_free_rate))   

r1 = pd.Series(r1, index=metrics)
metrics_df = metrics_df.append(r1, ignore_index=True)

In [24]:
# P2

r2 = np.zeros(len(metrics))
sb2 = stock_beta[new_cw2].copy()
pb2 = 0
twl2 = np.array(list(new_cw2.values()))
tcovdf2 = tdf2.cov()

pr2 = np.sum(tdf2.mean() * twl2) * days
pstd2 = np.sqrt(np.dot(twl2.T,np.dot(tcovdf2, twl2))) * np.sqrt(days)

r2[0] = pr2
r2[1] = pstd2

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r2[2] = (r2[0] - risk_free_rate) / r2[1]

# VaR => return - (Z * volatility)
r2[3] = abs(pr2 - (pstd2 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl2)):
    pb2 += twl2[j] * sb2[j]
r2[4] = pb2

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r2[5] = (r2[0] - risk_free_rate) / r2[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r2[6] = r2[0] - (risk_free_rate + r2[4] * (market_return - risk_free_rate))   

r2 = pd.Series(r2, index=metrics)
metrics_df = metrics_df.append(r2, ignore_index=True)

In [25]:
# DATAFRAME 

In [26]:
metrics_df.index = ['Portfolio 1', 'Portfolio 2']

In [27]:
weight_df = pd.DataFrame(columns=cw)
weight_df = weight_df.append(new_cw1, ignore_index=True)
weight_df = weight_df.append(new_cw2, ignore_index=True)

In [28]:
weight_df.index = ['Portfolio 1', 'Portfolio 2']

In [29]:
df

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,BHARTIARTL,BRITANNIA,CIPLA,...,RAMCOCEM,TORNTPOWER,TRENT,UNIONBANK,VGUARD,VBL,IDEA,VOLTAS,WHIRLPOOL,ZEEL
Date,,,,,,,,,,,,,,,,,,,,,
2016-07-08,201.571350,978.709473,540.462097,2259.038330,811.985352,2301.806152,279.636414,315.940796,1290.965820,509.883453,...,566.789124,153.016525,174.676132,129.699997,97.874008,NaN,61.065872,297.701111,824.691956,441.161804
2016-07-11,211.696152,994.299988,538.434570,2297.591309,847.569824,2354.808105,284.302429,320.212036,1315.910156,511.549377,...,581.450378,156.935471,174.577713,135.199997,98.574974,NaN,61.844162,299.547241,822.729065,442.476257
2016-07-12,212.231598,988.150940,554.852661,2299.976074,853.471008,2424.629883,281.103943,323.282013,1314.966675,505.375549,...,572.035706,155.154129,174.759796,137.850006,98.254150,NaN,62.442848,301.346069,861.301208,442.573578
2016-07-13,211.550110,977.547546,550.748169,2304.573242,848.341675,2357.503906,273.741180,323.638000,1319.753174,504.297516,...,568.554321,154.308014,174.174164,134.800003,97.410194,NaN,63.819824,307.499817,846.235535,444.959045
2016-07-14,211.160690,985.391052,554.110901,2315.869629,858.425659,2392.888428,277.654572,326.529999,1316.140137,501.700623,...,558.551208,152.882919,173.765656,140.850006,97.092842,NaN,64.418510,308.493927,846.726257,454.013977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-01,703.099976,3021.600098,746.450012,4064.005127,5957.523438,11813.585938,462.750000,522.650024,3595.800049,978.099976,...,1007.849976,458.299988,857.341309,38.700001,258.899994,723.750000,9.050000,1013.000000,2286.949951,215.250000
2021-07-02,710.400024,3005.000000,750.650024,4035.781494,5988.074219,11747.601562,462.750000,525.000000,3545.300049,979.200012,...,1009.450012,464.649994,899.213989,39.049999,264.899994,719.150024,8.800000,1013.900024,2265.750000,217.100006
2021-07-05,710.049988,3014.800049,760.349976,4061.057129,6062.903320,11920.208984,459.700012,523.950012,3523.550049,974.700012,...,1033.150024,468.899994,901.362610,38.900002,261.950012,713.150024,9.100000,1014.599976,2263.899902,218.850006


In [30]:
weight_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB
Portfolio 1,0.26313,0.03165,0.33668,0.00018,0.01454,0.02776,0.01332,NaN,0.01153,0.28391,0.01730,NaN
Portfolio 2,0.24072,0.01554,0.30410,NaN,0.00401,0.00021,0.00017,0.08996,NaN,0.24679,0.00722,0.09128


In [31]:
metrics_df

,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
Portfolio 1,0.900515,0.325621,2.580652,0.143007,0.892660,0.941361,0.760400
Portfolio 2,0.890909,0.312729,2.656319,0.163392,0.923608,0.899418,0.748023


In [32]:
df.to_csv('../data/StockDF.csv')
weight_df.to_csv('../data/WeightsDF.csv')
metrics_df.to_csv('../data/MetricsDF.csv')